# ORM using Peewee

ORM (Object Relational Mapper)  is a technique used in creating a "bridge" between object-oriented programs and, in most cases, relational databases.

Put another way, you can see the ORM as the layer that connects object oriented programming (OOP) to relational databases.

When interacting with a database using OOP languages, you'll have to perform different operations like creating, reading, updating, and deleting (CRUD) data from a database. By design, you use SQL for performing these operations in relational databases.

Class as defined in an Object Oriented (OO) programming language such as Python, is considered as non-scalar. It cannot be expressed as primitive types such as integers and strings.

On the other hand, databases like Oracle, MySQL, SQLite and others can only store and manipulate scalar values such as integers and strings organised within tables.

The programmer must either convert the object values into groups of scalar data types for storage in the database or convert them back upon retrieval, or only use simple scalar values within the program.

In an ORM system, each class maps to a table in the underlying database. Instead of writing tedious database interfacing code yourself, an ORM takes care of these issues, while you can focus on programming the logics of the system.

## Peewee

Peewee is a simple and small ORM. It has few (but expressive) concepts, making it easy to learn and intuitive to use.

* a small, expressive ORM
* python 2.7+ and 3.4+ (developed with 3.6)
* supports **sqlite**, **mysql**, **postgresql** and **cockroachdb**
* tons of extensions

In [22]:
# Import Library
from peewee import (
    SqliteDatabase, 
    Model, 
    TextField,
    IntegerField,
    DateTimeField)
import pandas as pd
import seaborn as sns

# Cleanup Database if exists
from pathlib import Path 
Path('database/sample_sqlite.db').unlink(missing_ok=True)

## Database
The Peewee `Database` object represents a connection to a database. The `Database` class is instantiated with all the information needed to open a connection to a database, and then can be used to:

* Open and close connections.
* Execute queries.
* Manage transactions (and savepoints).
* Introspect tables, columns, indexes, and constraints.

Peewee comes with support for SQLite, MySQL and Postgres. Each database class provides some basic, database-specific configuration options.

In [23]:
# Create Database via Peewee SqliteDatabase
sqlite_db = SqliteDatabase('database/sample_sqlite.db')
sqlite_db.connect()

True

## Create Table in SQLite

An object of Model sub class in Peewee API corresponds to a table in the database with which connection has been established. It allows performing database table operations with the help of methods defined in the Model class.

A user defined Model has one or more class attributes, each of them is an object of Field class. Peewee has a number of subclasses for holding data of different types. Examples are TextField, DatetimeField, etc. They correspond to the fields or columns in the database table. Reference of associated database and table and model configuration is mentioned in Meta class. Following attributes are used to specify configuration 

In [24]:
# Create Model that represent table
class Student(Model):
    name = TextField()
    age = IntegerField()
    
    class Meta:
        database = sqlite_db
        db_table = 'student'
        
# Create table using Model.create_table()
Student.create_table()

# This can also be done by using 
# sqlite_db.create_tables([Student])

Check whether user table is created

In [4]:
# Check table in database
!sqlite3 database/sample_sqlite.db .tables

student


Get Column name from student table to compare with class name

In [5]:
!sqlite3 database/sample_sqlite.db -cmd "PRAGMA table_info(student);" .exit

0|id|INTEGER|1||1
1|name|TEXT|1||0
2|age|INTEGER|1||0


Table student is created with same schema as Class Student


<br>

___

## Table Insertion

There are several method to insert new Rows to the table

In [25]:
# Insert new record using Object Creation with save() method
row1 = Student(name='John', age=20)
row1.save()

# Insert new record using Model Object with insert() method and execute()
row2 = Student.insert(name='Bob', age=22)
row2.execute()

# Insert new record using Model Object with create() method
Student.create(name='Jane', age=21)

<Student: 3>

### Bulk Inserts
In order to use multiple rows at once in the table, Peewee provides two methods: **bulk_create** and **insert_many**.

In [27]:
# Bulk Insert using Model.bulk_create()
a = Student(name='Alice', age=20)
b = Student(name='Ben', age=26)
Student.bulk_create([a, b])

In [28]:
# Bulk Insert using Model.insert_many()
rows = [
    {'name': 'Cindy', 'age': 25}, 
    {'name': 'David', 'age': 24}
]
Student.insert_many(rows).execute()

7

You can check now that those rows are already inserted

In [29]:
!sqlite3 database/sample_sqlite.db ".headers on" "SELECT * FROM student;"

id|name|age
1|John|20
2|Bob|22
3|Jane|21
4|Alice|20
5|Ben|26
6|Cindy|25
7|David|24


## Query Data using Peewee ORM

In [35]:
# Select all data from Student table to dataframe
pd.DataFrame(list(Student.select()))

,0
0,1
1,2
2,3
3,4
4,5
5,6
6,7


In [21]:
sqlite_db.close()

True

In [3]:
sns.get_dataset_names()

['anagrams',
 'anscombe',
 'attention',
 'brain_networks',
 'car_crashes',
 'diamonds',
 'dots',
 'dowjones',
 'exercise',
 'flights',
 'fmri',
 'geyser',
 'glue',
 'healthexp',
 'iris',
 'mpg',
 'penguins',
 'planets',
 'seaice',
 'taxis',
 'tips',
 'titanic']

In [4]:
sns.load_dataset('healthexp')

,Year,Country,Spending_USD,Life_Expectancy
0,1970,Germany,252.311,70.6
1,1970,France,192.143,72.2
2,1970,Great Britain,123.993,71.9
3,1970,Japan,150.437,72.0
4,1970,USA,326.961,70.9
...,...,...,...,...
269,2020,Germany,6938.983,81.1
270,2020,France,5468.418,82.3
271,2020,Great Britain,5018.700,80.4
272,2020,Japan,4665.641,84.7
